# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Задача:

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

# Описание данных

- Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Выгрузка данных

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбиение данных на выборки

In [5]:
df_train, df_valid_test = train_test_split(df, test_size = 0.4, random_state=12345, stratify=df['is_ultra'])
df_valid,df_test = train_test_split(df_valid_test, test_size = 0.5, random_state=12345, stratify=df_valid_test['is_ultra'])

# параметры и цели для обучения
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']

# параметры и цели для проверки модели
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

# параметры и цели для тестовой выборки
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

In [6]:
print(df_test.shape,df_valid.shape,df_train.shape)

(643, 5) (643, 5) (1928, 5)


## Исследовние модели

Главным критерием оценки качества модели для нас является параметр accuracy (показывает отношение правильных ответов модели к общему размеру данных). Для решения задачи классификации (использовать тариф Ultra, или нет) наиболее распространены три модели: Решающее дерево, Случайный лес и Логистическая регрессия. Из этой троицы самое самое низкое качество у решающего дерева, а самое высокое у случайного леса, при этом скорость алгоритма решающего дерева самая высокая, для случайного леса - самая низкая. Поэтому начнем с модели логистической регрессии. Она имеет средний показатель качества и высокую скорость исполнения. Если показатель accuracy нас не удовлетворит переидем к модели случайный лес. 


In [7]:
# LogisticRegression

model_log = LogisticRegression(random_state=12345)
model_log.fit(train_features,train_target) # обучение модели на тренировочной выборке
prediction = model_log.predict(valid_features)
accuracy_logist = accuracy_score(valid_target, prediction)

print(f'Accuracy = {accuracy_logist}')


Accuracy = 0.7387247278382582


Значение accuracy для данной модели оказалось слишком мало, попробуем модель Случайный лес

In [9]:
# RandomForestClassifier

#Для данной модели будем подбирать оптимальные гиперпараметры количества деревьев и глубины дерева 
best_accuracy = 0
model_nomber = None
model_depth = None
for est in range(1,11):
    for depth in range(1,11):
        model_rand = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model_rand.fit(train_features,train_target)
        prediction = model_rand.predict(valid_features)
        accuracy_rand_tree = accuracy_score(valid_target, prediction)
        if accuracy_rand_tree > best_accuracy:
            best_accuracy = accuracy_rand_tree
            model_nomber = est
            model_depth = depth
        else:    
            best_accuracy=best_accuracy
        print(f'Accuracy = {accuracy_rand_tree}')        
print(f'Best Accuracy in est {model_nomber}, depth {model_depth} = {best_accuracy}')        

Accuracy = 0.7480559875583204
Accuracy = 0.776049766718507
Accuracy = 0.776049766718507
Accuracy = 0.7480559875583204
Accuracy = 0.7713841368584758
Accuracy = 0.7791601866251944
Accuracy = 0.7433903576982893
Accuracy = 0.776049766718507
Accuracy = 0.7573872472783826
Accuracy = 0.76049766718507
Accuracy = 0.7153965785381027
Accuracy = 0.7573872472783826
Accuracy = 0.7900466562986003
Accuracy = 0.7698289269051322
Accuracy = 0.776049766718507
Accuracy = 0.7729393468118196
Accuracy = 0.7713841368584758
Accuracy = 0.776049766718507
Accuracy = 0.7791601866251944
Accuracy = 0.744945567651633
Accuracy = 0.7122861586314152
Accuracy = 0.7589424572317263
Accuracy = 0.7900466562986003
Accuracy = 0.7791601866251944
Accuracy = 0.7853810264385692
Accuracy = 0.7947122861586314
Accuracy = 0.7807153965785381
Accuracy = 0.7900466562986003
Accuracy = 0.7884914463452566
Accuracy = 0.7791601866251944
Accuracy = 0.7169517884914464
Accuracy = 0.776049766718507
Accuracy = 0.7900466562986003
Accuracy = 0.779160

Даже с двумя деревьями мы получили accuracy сильно больше необходимой. Лучший результат из 100 вариантов показало количество деревьев 6 и глубина дерева 6.

In [21]:
model_rand = RandomForestClassifier(random_state=12345, n_estimators=6, max_depth =6)
model_rand.fit(train_features,train_target)
prediction = model_rand.predict(train_features)
accuracy_rand_tree = accuracy_score(train_target, prediction)
print(f'Accuracy = {accuracy_rand_tree}') 

Accuracy = 0.8319502074688797


Проверим возможно нам подойдет дерево решений, ведь оно самое быстрое из предложенных моделей.


In [11]:
#DecisionTree

# найдем оптимальный гиперпараметр max_depth
best_accuracy = 0
model_nomber = None
for depth in range(1,21):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(train_features,train_target)
    prediction = model.predict(valid_features)
    accuracy_tree = accuracy_score(valid_target, prediction)
    if accuracy_tree > best_accuracy:
        best_accuracy = accuracy_tree
        model_nomber = depth
    else:    
        best_accuracy=best_accuracy
    print(f'Accuracy = {accuracy_tree}')        
print(f'Best Accuracy in est {model_nomber} = {best_accuracy}')         

Accuracy = 0.7402799377916018
Accuracy = 0.7729393468118196
Accuracy = 0.7776049766718507
Accuracy = 0.7542768273716952
Accuracy = 0.7853810264385692
Accuracy = 0.7744945567651633
Accuracy = 0.7869362363919129
Accuracy = 0.80248833592535
Accuracy = 0.7822706065318819
Accuracy = 0.7729393468118196
Accuracy = 0.7620528771384136
Accuracy = 0.7527216174183515
Accuracy = 0.744945567651633
Accuracy = 0.7558320373250389
Accuracy = 0.7480559875583204
Accuracy = 0.7387247278382582
Accuracy = 0.7153965785381027
Accuracy = 0.7076205287713841
Accuracy = 0.7045101088646968
Accuracy = 0.7060653188180405
Best Accuracy in est 8 = 0.80248833592535


Нашим требованиям удовлетворяет уже глубина дерева 1, чтобы accuracy наверняка была выше 0.75, примем за максимальную глубину дерева 3

In [12]:

model = DecisionTreeClassifier(random_state=12345, max_depth=8)
model.fit(train_features,train_target)
prediction = model.predict(train_features)
accuracy_train = accuracy_score(train_target, prediction)
print(f'Accuracy = {accuracy_train}')

Accuracy = 0.7909751037344398


### Вывод
Необходимое значение accuracy у модели "решающее дерево" и "случайный лес". Поскольку 'случайный' лес менее склонен к переобучению и показал лучшие результаты, для теста выбираем ее.

## Провка модели на тестовой выборке

Проверим модель случайного леса 

In [14]:
prediction = model_rand.predict(test_features)
accuracy_rand_test = accuracy_score(test_target, prediction)
print(f'Accuracy = {accuracy_rand_test}')

Accuracy = 0.7978227060653188


### Вывод
Значение для выбранной модели "случайного леса" Accuracy = 0.7962674961119751.
Нас это удовлетворяет.

##  Проверка модели на адекватность

Сравним тестовую выборки c baseline

In [16]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_features,train_target)
dummy_clf.score(valid_features,valid_target)


0.6936236391912908

### Вывод
Значение accuracy для baseline алгоритма оказалась меньше чем для модели "случайный лес". Тест на адекватность пройден.

## Вывод

Для поставленной задачи нам подходят две модели:
- Дерево решений
- Случайный лес

Логистическая регрессия показала значение accuracy меньше 0.75 

Accuracy тестовой выбороки для модели случайного леса равна 0,796. Тест на адекватность пройден, Выбираем модель "случайный лес"